In [1]:
!pip install surprise

     |████████████████████████████████| 6.5MB 5.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678549 sha256=c61435dcd02c698a9848a6bbdf277025ee5e02506c1e0ffbb80bcab762f7bc2e
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [2]:
!wget https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz

!wget https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz

!gunzip loc-gowalla_edges.txt.gz
!gunzip loc-gowalla_totalCheckins.txt.gz

--2020-04-11 17:13:01--  https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6351523 (6.1M) [application/x-gzip]
Saving to: ‘loc-gowalla_edges.txt.gz’

loc-gowalla_edges.t 100%[===================>]   6.06M  1.86MB/s    in 3.3s    

2020-04-11 17:13:05 (1.86 MB/s) - ‘loc-gowalla_edges.txt.gz’ saved [6351523/6351523]

--2020-04-11 17:13:06--  https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105470044 (101M) [application/x-gzip]
Saving to: ‘loc-gowalla_totalCheckins.txt.gz’

loc-gowalla_totalCh 100%[===================>] 100.58M  8.23MB/s    in 13s     

2020-04-11 17:13

##1.  Introduction
The  purpose  of  this  lab  is  to  implement  a  location-based  recommender  system  from  regular location-based social network (LBSN) data. Usually LBSN data give precise information regarding the locations visited by the users. These data also reveal who are the friends of the users. These two  pieces of information  (geographical  +  social)  can  be  exploited  into  a  standard  collaborative-filtering approach. At the end of this lab you should have all the building blocks of the model iGSLR, a geo-social location recommendation model. We suggest you to use the Surprise library as a main framework to implement the model. In this lab the part 2 has to be first. Then parts 3 and 4 can be achieved in parallel. Finally the part 5 combine the results of previous parts.

## 2.  Processing of the Dataset

The input dataset for this lab was collected from Gowalla, a popular LBSN that is often used to test recommendation methods with geographical dimensions. In practice, the dataset contains user profiles, user friendship, location profiles, and users’ check-in history made before June 1, 2011. It contains 36,001,959 check-ins made by 407,533 users over 2,724,891 locations. You can download the dataset from here. First of all, you have to extract the dataset and load it into a pandas dataframe. Filter users who have less than 5 check-ins in total. Associate for each user the list of his friends.  Put the result in a new dataframe df_user_friends. In the same way, associate for each user the list of the successive locations he visited. Put the result in a new dataframe df_user_locations.Then you have to compute for each pair(user, location) its corresponding visit frequency in order to build a new df_frequencies dataframe. Finally, transform and update the frequencies from df_frequencies into the range [0,10] with the following normalization transformation:

$$x\rightarrow 10. \text{  tanh  } [10.\frac{x-f_{min}}{f_{max}-f_{min}}]$$

where $f_{min}$  and $f_{max}$ refers  respectively  to  the  minimum  and  maximum  visit  frequencies  of  the dataset. This transformation is necessary in order to take into account of the long tail. You can then load df_frequencies into the Surprise framework with the load_from_df() function. Finally use the train_test_split function to divide df_frequencies into a 75 % train set, and 25 % test set.

In our experiments, the Gowalla datasets evaluate our proposed method . Gowalla was launched in 2007 and closed in 2012, which provides the location and social services that enable users to check in at places and make friends based on where they have visited. The Gowalla dataset (https://snap.stanford.edu/data/loc-gowalla.html) we used was crawled by its API from February 2009 to October 2010. By filtering out those users with fewer than 15 check-in POIs and those POIs with fewer than 10 visitors, we can reach our final dataset, which contains 25,379 users, 32,623 POIs, 1,395,856 check-ins, and 118,717 social ties.

In [3]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time
import math
from collections import defaultdict
import scipy.sparse as sparse

import numba
print(numba.__version__)
from numba import njit, jit
from numba import jitclass, types, typed
from tqdm import tqdm

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split

0.48.0


In [0]:
# Credits https://github.com/yangji9181/PACE2017/issues/4
def filter_data(checkin):
    poi_data = {}
    usr_data = {}
    for i in checkin:
        data = i.split('\t')
        uid = int(data[0])
        poi = int(data[-1])
        if not (poi in poi_data):
            poi_data[poi] = set()
        if not (uid in usr_data):
            usr_data[uid] = set()
        poi_data[poi].add(uid)
        usr_data[uid].add(poi)
    # filter out POIs less than 10 users
    poi_data = dict([(x, poi_data[x]) for x in poi_data if len(poi_data[x]) >= 10])
    # filter out users less than 15 check-ins
    usr_data = dict([(x, usr_data[x]) for x in usr_data if len(usr_data[x]) >= 15])
    new_checkin = []
    for i in checkin:
        data = i.split('\t')
        if int(data[0]) in usr_data and int(data[-1]) in poi_data:
            new_checkin.append(i)
    return new_checkin, poi_data, usr_data




In [5]:
checkin_file_path = 'loc-gowalla_totalCheckins.txt'
edge_file_path = 'loc-gowalla_edges.txt'
with open(checkin_file_path, 'r') as f:
        checkin = f.readlines()
checkin_len = len(checkin)
print('Loaded %d check-ins' % checkin_len)

Loaded 6442892 check-ins


In [6]:
print('STEP 1: Filtering check-ins')
itr = 1
while True:
        new_checkin, poi_data, usr_data = filter_data(checkin)
        new_checkin_len = len(new_checkin)
        print('Iteration %d: %d -> %d check-ins' % (itr, checkin_len, new_checkin_len))
        itr += 1
        checkin = new_checkin
        if new_checkin_len == checkin_len:
            break
        checkin_len = new_checkin_len
print("#User: %d, #POI: %d" % (len(usr_data), len(poi_data)))

STEP 1: Filtering check-ins
Iteration 1: 6442892 -> 2017717 check-ins
Iteration 2: 2017717 -> 1640213 check-ins
Iteration 3: 1640213 -> 1450654 check-ins
Iteration 4: 1450654 -> 1382891 check-ins
Iteration 5: 1382891 -> 1339690 check-ins
Iteration 6: 1339690 -> 1317671 check-ins
Iteration 7: 1317671 -> 1303115 check-ins
Iteration 8: 1303115 -> 1294859 check-ins
Iteration 9: 1294859 -> 1289178 check-ins
Iteration 10: 1289178 -> 1286049 check-ins
Iteration 11: 1286049 -> 1283417 check-ins
Iteration 12: 1283417 -> 1281840 check-ins
Iteration 13: 1281840 -> 1280413 check-ins
Iteration 14: 1280413 -> 1279723 check-ins
Iteration 15: 1279723 -> 1279019 check-ins
Iteration 16: 1279019 -> 1278754 check-ins
Iteration 17: 1278754 -> 1278530 check-ins
Iteration 18: 1278530 -> 1278448 check-ins
Iteration 19: 1278448 -> 1278378 check-ins
Iteration 20: 1278378 -> 1278315 check-ins
Iteration 21: 1278315 -> 1278274 check-ins
Iteration 22: 1278274 -> 1278274 check-ins
#User: 18737, #POI: 32510


In [0]:
user_id={}
loc_id={}
idx=0
idx2=0

In [0]:
with open('visited_spots.txt', 'w') as f:
      for uid_ in usr_data:
          
           if int(uid_)  in user_id.keys():
             uid=user_id[int(uid_)]
           else:
             user_id[int(uid_)]=int(idx)
             idx+=1
             uid=user_id[int(uid_)]

           for lid_ in list(usr_data[uid_]):
             if int(lid_)  in loc_id.keys():
               lid=loc_id[int(lid_)]
             else:
               loc_id[int(lid_)]=int(idx2)
               idx2+=1
               lid=loc_id[int(lid_)]

             f.write('%d %s\n' % (uid, str(lid)))

In [9]:
print('STEP 3: Generating user_network.txt')
with open(edge_file_path, 'r') as f:
        edges = f.readlines()
usr_graph = {}
for l in edges:
        data = l.split('\t')
        usr_a = int(data[0])
        usr_b = int(data[1])
        if not (usr_a in usr_graph):
            usr_graph[usr_a] = set()
        if not (usr_b in usr_graph):
            usr_graph[usr_b] = set()
        usr_graph[usr_a].add(usr_b)
        usr_graph[usr_b].add(usr_a)
with open('user_network.txt', 'w') as f:
        for uid_ in usr_graph:
           if int(uid_)  in user_id.keys():
             uid=user_id[int(uid_)]
             for vid_ in list(usr_graph[uid_]):
              if int(vid_)  in user_id.keys():
                vid=user_id[int(vid_)]
                f.write('%d %s\n' % (uid, str(vid)))


STEP 3: Generating user_network.txt


In [10]:
print('STEP 4: Generating spot_location.txt')
poi_loc = {}
for l in checkin:
        data = l.split('\t')
        poi = int(data[-1])
        if poi in poi_data and not (poi in poi_loc):
            lat = float(data[2])
            lon = float(data[3])
            poi_loc[poi] = (lat, lon)
with open('spot_location.txt', 'w') as f:
        for poi_ in poi_loc:
          if int(poi_) in loc_id.keys():
            poi=loc_id[int(poi_)]
            f.write('%d %f %f\n' % (poi, poi_loc[poi_][0], poi_loc[poi_][1]))

STEP 4: Generating spot_location.txt


In [0]:
checkins = pd.read_csv('visited_spots.txt', sep=" ", header=None)
checkins.columns = ["userid", "placeid"]

In [0]:
friendship = pd.read_csv('user_network.txt', sep=" ", header=None)
friendship.columns = ["userid1", "userid2"]

In [0]:
locations = pd.read_csv('spot_location.txt', sep=" ", header=None)
locations.columns = ["lid", "lat", "lng"]

In [0]:
df_frequencies=checkins.groupby(["userid", "placeid"]).size().reset_index(name="frequencie")

In [0]:
x=df_frequencies["frequencie"]
f_min=np.min(df_frequencies["frequencie"])
f_max=np.max(df_frequencies["frequencie"])

In [0]:
df_frequencies["frequencie"]=10*np.tanh(10*(x-f_min)/(f_max-f_min) )

In [0]:
df_frequencies, truth_data = train_test_split(df_frequencies, test_size=.25)

In [0]:
user_num, poi_num = 18737,32510


## 3. Geographical Computations

From df_user_locations you have access for every given user  u to the list $L_u$ of all locations he has been located to. Use this list to compute for each user the distances $d_{i,j}$ between each pair of locations of the list:$\forall l_i,l_j \in L_u ,d_{i,j}=\text{distance}(l_i,l_j)$. You will obtain a list D of distances. This list of distances can be used to 
compute the density $\hat{f}_u$ of any new given distance as follows:
$$\hat{f}_u(d_{ij})=\frac{1}{\left | D \right |}\sum_{{d}' \in D }K(\frac{d_{ij}-{d}'}{h})$$

where K(·) is the normal kernel function with fixed bandwidth.
$$K(x)=\frac{1}{\sqrt{2 \pi}}e^{\frac{-x^{2}}{2}}$$
with $h=1,06 \hat{\sigma}n^{\frac{-1}{5}}$ (n is  the  number  of  locations  in $L_u$, $\hat{\sigma}$ is  the  standard  deviation  of D).

The density  $\hat{f}_u$  will be used to estimate the likelihood that a new unvisited location $l_i$ matches the  user  geographical  preferences  based  on  his  previously  visited  locations.  This  is  achieved  by computing the distances between $l_i$ and every locations of $L_u=\{l_1,...,l_n\}$, and then estimating the likelihood of each distance given the distribution of the user. Finally we can take the empirical mean probability $p(l_i|L_u)$ of any new location for any given user:
$$p(l_i|L_u)=\frac{1}{n}\sum^{n}_{j=1}\hat{f}_u (d_{ij}) $$

This  probability  encodes  the  geographical  likelihood  that  user u will  visit  location $l_i$.  You  can first implement Equation 2 into a function density(). Then you can put Equation 3 into another function geo_proba() that will take a user’s list of visited locations and a new location as input,and return the probability of this new location as output.

In [0]:
def dist(loc1, loc2):
    lat1, long1 = loc1[0], loc1[1]
    lat2, long2 = loc2[0], loc2[1]
    if abs(lat1 - lat2) < 1e-6 and abs(long1 - long2) < 1e-6:
        return 0.0
    degrees_to_radians = math.pi/180.0
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
    earth_radius = 6371
    return arc * earth_radius

In [0]:
loc_coord={}
for row in locations.values:
        lid,lat, lng = row[0:3]
        lid, lat, lng = int(lid), float(lat), float(lng)
        loc_coord[lid] = (lat, lng)

In [21]:
ctime = time.time()
print("Precomputing Sparse Training Matrix ...", )
ratings_matrix = sparse.dok_matrix((user_num+1, poi_num+1))
for row in df_frequencies.values:
            uid, lid, freq = row
            uid, lid, freq = int(uid), int(lid), float(freq)
            ratings_matrix[uid, lid] = freq    
            
print("Done. Elapsed time:", time.time() - ctime, "s")

Precomputing Sparse Training Matrix ...
Done. Elapsed time: 11.001446962356567 s


In [22]:
from tqdm.notebook import tqdm
L = defaultdict(list)
for uid in tqdm(range(ratings_matrix.shape[0])):
        L[uid] = [loc_coord[lid]  for lid in ratings_matrix[uid].nonzero()[1].tolist()]

In [23]:
d = defaultdict(list)
for i in tqdm(range(len(L))) :
    u=list(L.keys())[i]
    if len(L[u]) > 1:
        for i in range(len(L[u])):
            for j in range(i+1, len(L[u])):
                d[u].append(dist(L[u][i], L[u][j]))


In [0]:
h = {}
for u in d:
    if not d[u]:
        h[u] = 1.06 * np.std(d[u]) * (1.0 / len(d[u])**0.2)
       

In [0]:
def K(x):
        return 1.0 / np.sqrt(2 * math.pi) * np.exp(-(x**2)/2)

def f(dij, u):
        return 1.0 * np.sum([K((dij - d) /h[u]) for d in d[u]]) / len(d[u]) / h[u]

def KernelDensityEstimation(u, lj):
        if u in h and not h[u] == 0:
            lat_j, lng_j = loc_coord[lj]
            return np.sum([f(dist((lat_i, lng_i), (lat_j, lng_j)), u) for lat_i, lng_i in L[u] ])  / len(L[u])
        return 1.0


## 4.  Social computations

Every user $u_i$ is associated with his set $F(u_i)$ of friends. From df_user_friends you can get the set $F(u_i)$. For every pair of users $(u_i,u_k)$, we can compute their social similarity score with the Jaccard coefficient as follows:

$$\text{Sim}(u_i,u_k)=\frac{\left |F(u_i) \cap F(u_k)  \right |}{\left | F(u_i) \cup  F(u_k)   \right |}$$

Implement this coefficient into a function of two arguments social_similarity(). Then this score can be exploited into the standard collaborative filtering model:
$$\hat{r}_{ij}=\frac{\sum _{u_k \in F(u_i) r_{k,j} \text{Sim}(u_i,u_k)}}{\sum _{u_k \in F(u_i)  \text{Sim}(u_i,u_k)}}$$

Finally we cant to transform the prediction  $\hat{r}_{ij}$ into a probability as follows:
$$\hat{p}_{ij}=\frac{\hat{r}_{ij}}{max_{l_j \in L \setminus  L_j} {\hat{r}_{ij}}}$$


In [0]:
def Sim(u_i,u_k):
      Fu_i=friendship[friendship.userid1==u_i].values[:,1]
      Fu_k=friendship[friendship.userid1==u_k].values[:,1]
      intersection = len(list(set(Fu_i).intersection(Fu_k)))
      union = (len(Fu_i) + len(Fu_k)) - intersection
      return float(intersection) / union    

In [27]:
social_proximity=defaultdict(list)

for i  in tqdm(range(len(friendship.values))):
            uid1, uid2=friendship.values[i]
            social_proximity[uid1].append([uid2, Sim(uid1, uid2)])
            social_proximity[uid2].append([uid1, Sim(uid1, uid2)])
      

In [0]:
def FriendBasedCF(i, j):
        if i in social_proximity:
            numerator = np.sum([weight * ratings_matrix[k, j] for k, weight in social_proximity[i]])
            denominator = np.sum([weight for k, weight in social_proximity[i]])
            return numerator / denominator
        return 0.0

## 5.  Generate & Test Recommendations
The goal is to generate a recommendation score $\hat{s}_{ij}$ for a given user i and a given location j. This recommendation score can be computed with equations 3 and 5 as follows :
$$\hat{s}_{ij}=\frac{\hat{p}_{ij}+p(l_i \mid L_u)}{2}$$

Generate recommendations for each user in the test set and get the precision@K and recall@K for K∈ $\{5,10,15 \}$. To implement Equation 6 you have to create a new prediction algorithm as youcan see here. You have to create aGSLRclass (i.e.Geographical Social Location Recommendation)and populate afit()and aestimate()functions. These functions will of course depend on thefunctions you have implemented in parts 3 and 4. To compute the recall and precision, you canreproduce this part of the documentation. You can also use cross validation as shown here.

In [0]:
def precisionk(actual, predicted):
    return 1.0 * len(set(actual) & set(predicted)) / len(predicted)


def recallk(actual, predicted):
    return 1.0 * len(set(actual) & set(predicted)) / len(actual)

In [0]:
ground_truth = defaultdict(set)
for row in truth_data.values:
        uid, lid, _ = row
        uid, lid = int(uid), int(lid)
        ground_truth[uid].add(lid)

In [0]:
training_tuples=set([(int(uid), int(lid)) for uid, lid in df_frequencies.values[:,0:2]])

In [0]:
from tqdm.notebook import tqdm

all_uids = list(range(user_num))
all_lids = list(range(poi_num))
np.random.shuffle(all_uids)


precision_k5, recall_k5 = [], []
precision_k10, recall_k10 = [], []
precision_k15, recall_k15 = [], []

for i in tqdm(range(len(all_uids))):
        uid=all_uids[i]
        if uid in ground_truth:
            overall_scores = [FriendBasedCF(uid, lid) * KernelDensityEstimation(uid, lid)
                              if (uid, lid) not in training_tuples else -1
                              for lid in all_lids]
               
            overall_scores = np.array(overall_scores)

            predicted_k5 = list(reversed(overall_scores.argsort()))[:5]
            predicted_k10 = list(reversed(overall_scores.argsort()))[:10]
            predicted_k15 = list(reversed(overall_scores.argsort()))[:15]
            actual = ground_truth[uid]

            precision_k5.append(precisionk(actual, predicted_k5[:5]))
            recall_k5.append(recallk(actual, predicted_k5[:5]))

            precision_k10.append(precisionk(actual, predicted_k10[:10]))
            recall_k10.append(recallk(actual, predicted_k10[:10]))

            precision_k15.append(precisionk(actual, predicted_k15[:15]))
            recall_k15.append(recallk(actual, predicted_k15[:15]))

            
            